In [1]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
import os
import getpass

In [2]:
load_dotenv()

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google Gemini uchun API kalitni kiriting")

# Init a model
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")
print("Model bilan aloqa bo'glandi")

Model bilan aloqa bo'glandi


In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="I want to opan a resturant for uzbek food. Suggest a fancy name for this. Only one name")]).content

"**The Emir's Table**"

# LangChain PromptTemplate

In [4]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to opan a resturant for {cuisine} food. Suggest a fancy name for this. Only one name"
)

prompt_template_name.format(cuisine="Uzbek")

'I want to opan a resturant for Uzbek food. Suggest a fancy name for this. Only one name'

# Chains

### LLMChain

In [5]:
from langchain.chains import LLMChain

chain = LLMChain(llm=model, prompt=prompt_template_name)
chain.run("American")

C:\Users\User\AppData\Local\Temp\ipykernel_19872\558854879.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=model, prompt=prompt_template_name)
C:\Users\User\AppData\Local\Temp\ipykernel_19872\558854879.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("American")


'The Sovereign Table'

In [6]:
# We might be see some warnings. Now we can try new way. LLMChain is not needed

chain = prompt_template_name | model
chain.invoke("Uzbek").content

'The Azure Courtyard'

### SimpleSequentialChain

In [7]:
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template = "I want to opan a resturant for {cuisine} food. Suggest a fancy name for this. Only one name"
)

new_chain = LLMChain(llm=model, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables=['resturant_name'],
    template = "Suggest some menu items for {resturant_name}. Return it as a comma separated list.")

food_item_chain = LLMChain(llm=model, prompt=prompt_template_items)


In [8]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[new_chain, food_item_chain])
response = chain.run("Uzbek")
print(response)

Mutton Biryani, Chicken Pulao, Mutton Qorma, Chicken Karahi, Nihari, Haleem, Seekh Kebab, Chapli Kebab, Daal Makhani, Palak Paneer, Assorted Naan, Sheermal, Raita, Kachumber Salad, Gulab Jamun, Shahi Tukda, Kheer, Lassi


### SequentialChain

In [9]:
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to opan a resturant for {cuisine} food. Suggest a fancy name for this. Only one name"
)

name_chain = LLMChain(llm=model, prompt=prompt_template_name, output_key="resturant_name")

prompt_template_items = PromptTemplate(
    input_variables=['resturant_name'],
    template="Suggest some menu items for {resturant_name}. Return it as a comma separated list."
)
food_items_chain = LLMChain(llm=model, prompt=prompt_template_items, output_key="menu_items")

In [10]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['resturant_name', 'menu_items']
)
chain({'cuisine' : "Uzbek"})   

C:\Users\User\AppData\Local\Temp\ipykernel_19872\3204647046.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({'cuisine' : "Uzbek"})


{'cuisine': 'Uzbek',
 'resturant_name': '**Ipak Saroy**',
 'menu_items': 'Ipak Saroy Signature Plov, Lamb Shashlik, Tandir Samsa (Lamb & Pumpkin), Hand-Pulled Lagman, Steamed Manti, Uzbek Shurpa (Lamb & Vegetable Soup), Achichuk Salad, Kazan Kabob, Dimlama (Steamed Meat & Vegetable Stew), Samarkand Halva, Assorted Dried Fruits & Nuts Platter, Tandir Non (Uzbek Bread), Green Tea (Choy)'}

# Memory

In [11]:
chain = LLMChain(llm=model, prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)

El Encanto


In [ ]:
name = chain.run("Uzbek")

In [13]:
name

'Samarkand Gold'

In [14]:
dir(chain)

['InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_pri

In [15]:
chain.memory

In [16]:
type(chain.memory)

NoneType

In [17]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

chain = LLMChain(llm=model, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)

C:\Users\User\AppData\Local\Temp\ipykernel_19872\2743924038.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


Obsidian Agave


In [18]:
print(chain.memory.buffer)

Human: Mexican
AI: Obsidian Agave


In [19]:
name = chain.run("Russian")
print(name)

Nevsky


In [20]:
print(chain.memory.buffer)

Human: Mexican
AI: Obsidian Agave
Human: Russian
AI: Nevsky


In [21]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=model)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


C:\Users\User\AppData\Local\Temp\ipykernel_19872\997297751.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  convo = ConversationChain(llm=model)


In [22]:
convo.run("Who won the first cricket world cup?")

"Oh, that's a fantastic question about cricket history! The very first Cricket World Cup was held in **1975**, and it was won by the mighty **West Indies** team!\n\nThey were an absolutely dominant force in those early days. The tournament was hosted in England, and it was the first time an international cricket competition of that scale was played in the One Day International (ODI) format, with 60 overs per side.\n\nIn that inaugural final, played at the iconic Lord's Cricket Ground, the West Indies faced off against Australia. The West Indies, captained by the legendary Clive Lloyd, put up a fantastic score, with Lloyd himself playing an incredible innings of 102 runs off just 85 balls. They ended up defending their total successfully, beating Australia by 17 runs.\n\nIt was a truly historic event that set the stage for all the World Cups that followed! What a way to kick off such an important tournament!"

In [24]:
convo.run("What is 5+5? Write short answer")

"It's **ten**! Keeping it short and sweet, just as you asked!"

In [26]:
convo.run(" Who was the captain of the winning team? Write a short answer")

'It was **Clive Lloyd**!'

In [27]:
convo.memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Who won the first cricket world cup?', additional_kwargs={}, response_metadata={}), AIMessage(content="Oh, that's a fantastic question about cricket history! The very first Cricket World Cup was held in **1975**, and it was won by the mighty **West Indies** team!\n\nThey were an absolutely dominant force in those early days. The tournament was hosted in England, and it was the first time an international cricket competition of that scale was played in the One Day International (ODI) format, with 60 overs per side.\n\nIn that inaugural final, played at the iconic Lord's Cricket Ground, the West Indies faced off against Australia. The West Indies, captained by the legendary Clive Lloyd, put up a fantastic score, with Lloyd himself playing an incredible innings of 102 runs off just 85 balls. They ended up defending their total successfully, beating Australia by 17 runs.\n\nIt was a truly histor

In [28]:
convo.memory.buffer

"Human: Who won the first cricket world cup?\nAI: Oh, that's a fantastic question about cricket history! The very first Cricket World Cup was held in **1975**, and it was won by the mighty **West Indies** team!\n\nThey were an absolutely dominant force in those early days. The tournament was hosted in England, and it was the first time an international cricket competition of that scale was played in the One Day International (ODI) format, with 60 overs per side.\n\nIn that inaugural final, played at the iconic Lord's Cricket Ground, the West Indies faced off against Australia. The West Indies, captained by the legendary Clive Lloyd, put up a fantastic score, with Lloyd himself playing an incredible innings of 102 runs off just 85 balls. They ended up defending their total successfully, beating Australia by 17 runs.\n\nIt was a truly historic event that set the stage for all the World Cups that followed! What a way to kick off such an important tournament!\nHuman: What is 5+5?\nAI: Oh, 

In [29]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=1) 

convo = ConversationChain(llm=model, memory=memory)

convo.run("Who won the first cricket world cup?")

C:\Users\User\AppData\Local\Temp\ipykernel_19872\1528840562.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


"Oh, that's a fantastic question about cricket history! The very first Cricket World Cup, officially known as the **Prudential Cup**, was won by the mighty **West Indies**!\n\nIt took place in **1975**, and it was hosted by **England**. Eight teams participated in that inaugural tournament: England, India, Australia, New Zealand, Pakistan, Sri Lanka, East Africa, and, of course, the West Indies.\n\nThe final was played on June 21st, 1975, at the iconic Lord's Cricket Ground in London. The West Indies faced off against **Australia**. It was quite a thrilling match! The West Indies batted first and scored 291 runs for 8 wickets in their 60 overs (yes, it was a 60-over-a-side match back then, not 50 like today!). Their captain, the legendary **Clive Lloyd**, played an incredible innings, scoring 102 runs off just 85 balls, which was a truly blistering knock for that era.\n\nIn response, Australia fought hard, with Alan Turner scoring 40 and Gary Gilmour taking 5 wickets earlier in the Wes

In [30]:
convo.run("What is 5+5? Write short answer")

"Oh, that's a quick one! The answer to 5 + 5 is **10**!"

In [31]:
convo.run(" Who was the captain of the winning team? Write a short answer")

'Oh, that\'s an interesting question! Based on our conversation so far, I don\'t have any information about a "winning team" or who their captain might have been. We\'ve only talked about math, specifically 5 + 5! So, I\'m afraid I don\'t know the answer to that one.'